In [1]:
import scipy as sc
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

def morlet(fecx,wide,freqm):
    frecm=freqm
    twav = np.linspace(-2,2,frecm*4)
    #Sine wave
    com_sin = np.exp(1j*2*np.pi*fecx*twav)
    #Gauss wave
    sigma = wide/(2*np.pi*fecx)
    gauss_c = np.exp(-twav**2/(2*sigma**2))
    #Normalization
    A_sigma = 1/(np.sqrt(sigma*np.sqrt(np.pi)))
    #Morlet wavelet
    wavelet = com_sin*gauss_c
    return wavelet

def MRA(sig,fi,ff,freq_int,wavelet,freqm):
    #vect freq var
    range_cycles = [1,10] #barrido de número de ciclos
    delta = ff-fi #intervalo de frecuencias de descomposición
    num_frq = int(delta*(1/freq_int)) # número de frecuencias en que se descompone
    vfrec = np.linspace(fi,ff,num_frq) #vector de frecuencias
    num_cycles =np.logspace(np.log10(range_cycles[0]), np.log10(range_cycles[-1]),num_frq) #vector de ciclos
    #Preparation
    n,m = len(sig),len(wavelet(vfrec[0],range_cycles[0],freqm))
    p = n+m-1
    k = num_frq
    half_wav = int(np.floor(m/2))
    matrx_pad = np.zeros([k,p])
    sig_fft = sc.fft.fft(np.pad(sig,(0,p-n+1),'constant'))
    #Convolution
    conv_mat = np.zeros([k,n])
    aux_vec = []
    for i in range(k):
        wave = np.pad(wavelet(vfrec[i],num_cycles[i],freqm), (0,n),'constant')
        wave_fft = sc.fft.fft(wave) 
        wave_fft = wave_fft / wave_fft.max()
        conv_res_prim = np.multiply(sig_fft,wave_fft)
        conv_res = abs(sc.fft.ifft(conv_res_prim))**2 #resultado 
        conv_res = conv_res[half_wav:-half_wav] #recorte del padding
        for j in range(n):
            conv_mat[i][j] = conv_res[j]
    return conv_mat